In [60]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import sklearn.preprocessing

In [61]:
train_file = "../data/train.csv"
test_file = "../data/test.csv"


train_data_raw = pd.read_csv(train_file)
test_data_raw = pd.read_csv(test_file)


In [62]:

### CLEAN DATA FUNC

def clean_func(train_data):
    
    ## DO IMPUTATION 
    # FARE
    imp_fare = Imputer(missing_values="NaN", strategy="mean")
    imp_fare.fit(train_data[["Fare"]])
    train_data[["Fare"]]=imp_fare.transform(train_data[["Fare"]]).ravel() 

    # Age
    imp=Imputer(missing_values="NaN", strategy="mean")
    imp.fit(train_data[["Age"]])
    train_data[["Age"]]=imp.transform(train_data[["Age"]]).ravel() 
    
    # Filna
    train_data["Cabin"] = train_data["Cabin"].fillna("")

    
    # one hot encoding
    sex_features = pd.get_dummies(train_data["Sex"])
    embarked_features = pd.get_dummies(train_data["Embarked"])
    train_data_extras = pd.concat([train_data,sex_features,embarked_features],axis=1)

    # HACK - REMOVE T WHICH IS NOT IN TEST LIKELY ERRROR 
    cabin_letters = pd.get_dummies(train_data['Cabin'].map(lambda x: "empty" if len(x)==0 or x[0]=="T" else x[0]))

#    cabin_letters = pd.get_dummies(train_data['Cabin'].map(lambda x: "empty" if len(x)==0 else x[0]))
    cabin_letters.columns = ["Cabin_letter_"+i for i in cabin_letters.columns]
    train_data_extras = pd.concat([train_data_extras,cabin_letters],axis=1)
    

    train_data_extras["Cabin_number"] = train_data['Cabin'].map(lambda x: -99 if len(x)==0 else x.split(" ")[0][1:]) 

    return train_data_extras



In [63]:
train_data = clean_func(train_data_raw)


In [64]:
### NULL HANDLE
imp=Imputer(missing_values="NaN", strategy="mean")
imp.fit(train_data[["Age"]])

train_data[["Age"]]=imp.transform(train_data[["Age"]]).ravel() # what is ravel???

train_data["Cabin"] = train_data["Cabin"].fillna("")




In [65]:
train_data.head(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,S,Cabin_letter_A,Cabin_letter_B,Cabin_letter_C,Cabin_letter_D,Cabin_letter_E,Cabin_letter_F,Cabin_letter_G,Cabin_letter_empty,Cabin_number
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,...,1,0,0,0,0,0,0,0,1,-99
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,...,0,0,0,1,0,0,0,0,0,85
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,...,1,0,0,0,0,0,0,0,1,-99
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,...,1,0,0,1,0,0,0,0,0,123
4,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,...,1,0,0,0,0,0,0,0,1,-99
5,6,0,3,"Moran, Mr. James",male,29.699118,0,0,330877,8.4583,...,0,0,0,0,0,0,0,0,1,-99
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.000000,0,0,17463,51.8625,...,1,0,0,0,0,1,0,0,0,46
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.000000,3,1,349909,21.0750,...,1,0,0,0,0,0,0,0,1,-99
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.000000,0,2,347742,11.1333,...,1,0,0,0,0,0,0,0,1,-99
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.000000,1,0,237736,30.0708,...,0,0,0,0,0,0,0,0,1,-99


In [66]:
# LOOK AT UNIQUE VALUES
text_col = ["Sex","Ticket","Cabin","Embarked"]

for i in text_col:
    print(i,train_data[i].unique()[0:20])

## Sex, Embarked can be 1 hot encoded



('Sex', array(['male', 'female'], dtype=object))
('Ticket', array(['A/5 21171', 'PC 17599', 'STON/O2. 3101282', '113803', '373450',
       '330877', '17463', '349909', '347742', '237736', 'PP 9549',
       '113783', 'A/5. 2151', '347082', '350406', '248706', '382652',
       '244373', '345763', '2649'], dtype=object))
('Cabin', array(['', 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6', 'C23 C25 C27',
       'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33', 'F G73', 'E31',
       'A5', 'D10 D12'], dtype=object))
('Embarked', array(['S', 'C', 'Q', nan], dtype=object))


In [67]:
# one hot encoding
sex_features = pd.get_dummies(train_data["Sex"])
embarked_features = pd.get_dummies(train_data["Embarked"])
train_data_extras = pd.concat([train_data,sex_features,embarked_features],axis=1)


In [68]:
## Advanced string processing
# Name, Cabin, ticket

## Remove all digits
#tmp = train_data['Cabin'].map(lambda x: "".join([i for i in x if not i.isdigit()]))


cabin_letters = pd.get_dummies(train_data['Cabin'].map(lambda x: "empty" if len(x)==0 else x[0]))
cabin_letters.columns = ["Cabin_letter_"+i for i in cabin_letters.columns]
train_data_extras = pd.concat([train_data_extras,cabin_letters],axis=1)


train_data_extras["Cabin_number"] = train_data['Cabin'].map(lambda x: -99 if len(x)==0 else x.split(" ")[0][1:]) 
# train_data_extras.head()

#["Cabin"].unique()


In [69]:
# Null handling
train_data.isnull().sum()


PassengerId           0
Survived              0
Pclass                0
Name                  0
Sex                   0
Age                   0
SibSp                 0
Parch                 0
Ticket                0
Fare                  0
Cabin                 0
Embarked              2
female                0
male                  0
C                     0
Q                     0
S                     0
Cabin_letter_A        0
Cabin_letter_B        0
Cabin_letter_C        0
Cabin_letter_D        0
Cabin_letter_E        0
Cabin_letter_F        0
Cabin_letter_G        0
Cabin_letter_empty    0
Cabin_number          0
dtype: int64

In [70]:
target = "Survived"

a = train_data.dtypes 
b = a[(a=="int64") | (a=="float64") | (a=="uint8")]
numerics = [i for i in b.index if i not in target]



print(numerics)
print(target)

['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'female', 'male', 'C', 'Q', 'Cabin_letter_A', 'Cabin_letter_B', 'Cabin_letter_C', 'Cabin_letter_D', 'Cabin_letter_E', 'Cabin_letter_F', 'Cabin_letter_G', 'Cabin_letter_empty']
Survived


In [71]:
train_data[numerics].head(3)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,Cabin_letter_A,Cabin_letter_B,Cabin_letter_C,Cabin_letter_D,Cabin_letter_E,Cabin_letter_F,Cabin_letter_G,Cabin_letter_empty
0,1,3,22.0,1,0,7.2500,0,1,0,0,0,0,0,0,0,0,0,1
1,2,1,38.0,1,0,71.2833,1,0,1,0,0,0,1,0,0,0,0,0
2,3,3,26.0,0,0,7.9250,1,0,0,0,0,0,0,0,0,0,0,1


In [72]:
X_train,X_test,Y_train,Y_test = train_test_split(train_data[numerics]
                              ,train_data[target].values
                              ,test_size=0.3
                              ,random_state=42)

In [73]:
log_reg = LogisticRegression(penalty="l2", dual=False, tol=0.0001, C=1.0
                             , fit_intercept=True, intercept_scaling=1
                             , class_weight=None, random_state=None
                             , solver="liblinear", max_iter=100
                             , multi_class="ovr", verbose=0
                             , warm_start=False, n_jobs=1)

log_reg.fit(X_train,Y_train)




LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [74]:
Y_pred = log_reg.predict(X_test)

In [75]:
metrics.accuracy_score(Y_test,Y_pred) 

0.80223880597014929

In [76]:
# get parameters
coef = list(log_reg.coef_.ravel())
intercept = log_reg.intercept_

# print them
print intercept
for id, i in enumerate(coef):
    print(numerics[id],i)


[ 1.3136486]
('PassengerId', 0.0002972427153843155)
('Pclass', -0.6074078790837637)
('Age', -0.029151477976472252)
('SibSp', -0.25137296791278096)
('Parch', -0.10465631435944094)
('Fare', 0.0045768814182350539)
('female', 1.9379891103387796)
('male', -0.62434050671729624)
('C', 0.63114782742811715)
('Q', 0.33968631813959643)
('Cabin_letter_A', -0.10532371825008076)
('Cabin_letter_B', 0.28048290184983515)
('Cabin_letter_C', -0.56490672994194313)
('Cabin_letter_D', 0.58817531061936945)
('Cabin_letter_E', 1.2505599344214455)
('Cabin_letter_F', 0.79142741651120896)
('Cabin_letter_G', -0.57283618300881445)
('Cabin_letter_empty', -0.3539303285795814)


# PREDICT AND STORE OUTPUT

In [77]:
### HACK TO COMPUTE TEST RESULT
test_data = clean_func(test_data_raw)

#test_data[["Age"]]=imp.transform(test_data[["Age"]]).ravel()

In [78]:
## DO IMPUTATION ON FARE
# imp_fare = Imputer(missing_values="NaN", strategy="mean")
# imp_fare.fit(train_data[["Fare"]])

# test_data[["Fare"]]=imp_fare.transform(test_data[["Fare"]]).ravel() # what is ravel???


In [79]:
test_data.isnull().sum()


PassengerId           0
Pclass                0
Name                  0
Sex                   0
Age                   0
SibSp                 0
Parch                 0
Ticket                0
Fare                  0
Cabin                 0
Embarked              0
female                0
male                  0
C                     0
Q                     0
S                     0
Cabin_letter_A        0
Cabin_letter_B        0
Cabin_letter_C        0
Cabin_letter_D        0
Cabin_letter_E        0
Cabin_letter_F        0
Cabin_letter_G        0
Cabin_letter_empty    0
Cabin_number          0
dtype: int64

In [80]:
#test_data_y = log_reg.predict(test_data)
test_data_y = log_reg.predict(test_data[numerics])


#train_data[numerics].head(3)

In [81]:
output = pd.DataFrame(zip(list(test_data["PassengerId"]),list(test_data_y)))
output.columns = ["PassengerId","Survived"]

In [82]:
output.to_csv(index=False, path_or_buf= "../data/output.csv")
